In [1]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from substitutions import appos
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
import numpy as np
import contractions
import copy

[nltk_data] Downloading package stopwords to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
data_dir = '../Humor,Hist,Media,Food'
file_names = os.listdir(data_dir)
file_paths = [(data_dir + '/' + fname) for fname in file_names]
docID_to_doc_mapping = {}
for i in range(len(file_names)):
    docID_to_doc_mapping[i] = file_names[i]

In [3]:

def read_files(fpaths):
    '''
        Reads the files and pre process every file's text to form word tokens for every file.
        Returns a 2-D list containing word tokens for every file
    '''
    file_tokens = []
    for fpath in fpaths:
        f = open(fpath, 'r', encoding='utf-8', errors='replace') #open the file
        ftxt_unprocessed = f.read() #read the text of the file
        ftoks = preprocess_file(ftxt_unprocessed) #preprocessing the text to form word tokens
        file_tokens.append(ftoks)
    return file_tokens

def check_alnum(tok):
    '''
        Remove non-alphanumeric characters from a string
    '''
    tok = ''.join(x for x in tok if x.isalnum() == True)
    return tok
def remove_punct(tok):
    '''
        Remove the punctuation in token
    '''
    punctuations = string.punctuation
    tok = ''.join(x for x in tok if x not in punctuations)
    return tok

def remove_space(tok):
    '''
        Remove the spaces in token
    '''
    tok = ''.join(x for x in tok if x != ' ')
    return tok

def preprocess_file(file_text):
    '''
        Preprocess the file text and converting to word tokens
        Input: string File text
        Returns file_tokens, word tokens for the pre processed text
    '''
    #converting text to lowercase
    text = file_text.lower()
    
    # print(f"Original tokens : \n{word_tokenize(text)}\n")
    # print("\n-------------------------\n")

    #Fixing the contractions
    text = contractions.fix(text)

    #omitting all non-alphanumeric characters
    # text = ''.join([x if (x.isalnum() == True or x == " ") else " " for x in text ])

    #Performing word tokenization
    all_toks = word_tokenize(text)
    all_unique_toks = set(all_toks)

    #Omitting all the non-alphanumeric characters in tokens
    all_unique_toks = [check_alnum(x) for x in all_unique_toks]

    # print(f"All unique tokens :\n{all_unique_toks}\n")
    # print("\n-------------------------\n")

    #removing stopwords
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in all_unique_toks if tok not in stop_words]
    # print(f"After removing stopwords :\n{file_toks}\n")
    # print("\n-------------------------\n")

    #removing punctations if any remain
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            toks_no_punct.append(ctok)
    # print(f"After removing puncts :\n{toks_no_punct}\n")
    # print("\n-------------------------\n")

    #removing spaces in any remain & check if it is a number
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            cleaned_toks.append(ctok)
    cleaned_toks = list(set(cleaned_toks))
    # print(f"After removing spaces and removing numbers :\n{cleaned_toks}\n")
    # print("\n-------------------------\n")
    #stemming tokens
    # stemmer = PorterStemmer()
    # stemmed_toks = set()

    # for tok in cleaned_toks:
    #     stok = stemmer.stem(tok)
    #     if(stok != "" or (stok not in stop_words)): #consider non-blank tokens and the ones which have length greater than 1
    #         stemmed_toks.add(stok)
    # print(f"After stemming and removing single letters :\n{stemmed_toks}\n")
    # print("\n-------------------------\n")
    final_tokens = [tok for tok in cleaned_toks]

    return final_tokens

def preprocess_query(query_text):
    '''
        Performs the pre processing of the query text
        Input: A string representing query text
        Output: Query word tokens
    '''
    text = query_text.lower()
    
    # print(f"Original Query Tokens : \n{word_tokenize(text)}\n")
    # print("\n-------------------------\n")
    
    #Fixing the contractions
    text = contractions.fix(text)

    #omitting all non-alphanumeric characters
    # text = ''.join([x if (x.isalnum() == True or x == " ") else " " for x in text ])

    #word tokenization
    all_toks = word_tokenize(text)
    all_unique_toks = []
    for tok in all_toks:
        if(tok not in all_unique_toks):
            all_unique_toks.append(tok)

    all_unique_toks = [check_alnum(x) for x in all_unique_toks]

    # print(f"All unique tokens :\n{all_unique_toks}\n")
    # print("\n-------------------------\n")
    #removing stopwords
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in all_unique_toks if tok not in stop_words]
    # print(f"After removing stopwords :\n{file_toks}\n")
    # print("\n-------------------------\n")
    #removing punctations if any remain
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            toks_no_punct.append(ctok)
    # print(f"After removing puncts :\n{toks_no_punct}\n")
    # print("\n-------------------------\n")

    #removing spaces in any remain & check if it is a number
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            cleaned_toks.append(ctok)
    
    final_tokens = []
    for tok in cleaned_toks:
        if(tok not in final_tokens):
            final_tokens.append(tok)
    # cleaned_toks = list(set(cleaned_toks))
    # print(f"After removing spaces and removing numbers :\n{cleaned_toks}\n")
    # print("\n-------------------------\n")
    #stemming tokens
    # stemmer = PorterStemmer()
    # stemmed_toks = set()

    # for tok in cleaned_toks:
    #     stok = stemmer.stem(tok)
    #     if(stok != "" and len(stok) > 1 and (stok not in stop_words)): #consider non-blank tokens and the ones which have length greater than 1
    #         stemmed_toks.add(stok)
    # print(f"After stemming and removing single letters :\n{stemmed_toks}\n")
    # print("\n-------------------------\n")
    # final_tokens = [tok for tok in cleaned_toks]

    return final_tokens



    

In [4]:
#read the files to get file tokens for each and every file
file_toks = read_files(file_paths)

In [5]:
def create_inverted_index(file_toks):
    ''' 
        Create the inverted index based on the file tokens
        Input: A 2 dimensional list containg word tokens for every file
    '''
    inv_index_postings = {}
    inv_index_frequency = {}
    for i in range(len(file_toks)):
        for tok in file_toks[i]:
            if(tok not in inv_index_postings.keys()):
                inv_index_postings[tok] = [i]
                inv_index_frequency[tok] = 1
            else:
                inv_index_postings[tok].append(i)
                inv_index_frequency[tok] += 1
    inv_index_postings = dict(sorted(inv_index_postings.items()))
    terms_list = inv_index_postings.keys()
    for word in terms_list:
        inv_index_postings[word].sort()
    return inv_index_postings, inv_index_frequency

In [6]:
def getDocsFromID(docID_to_docs, doc_IDs):
    '''
        Given a list of document IDs, it outputs the document names corresponding to thos IDs.
        Input: docID_to_docs (mapping between docID -> doc_name), docIDs - list of input document IDs
        Returns: doc_names - list of doc_names corresponding to document IDs in doc_IDs
    '''
    doc_names = []
    for doc_ID in doc_IDs:
        doc_names.append(docID_to_docs[doc_ID])
    return doc_names

In [8]:
def retreive_postings(inv_index_postings, term):
    '''
        Given a term, retreive its posting list.
        Input: inv_index_postings - inverted index postings, term
        Returns: [] if term not in index, posting list for the term otherwise
    '''
    terms_list = inv_index_postings.keys()
    if(term not in terms_list):
        return []
    else:
        posting = inv_index_postings[term]
        return posting

In [9]:
def retreive_frequency(inv_index_frequency, term):
    terms_list = inv_index_frequency.keys()
    if(term not in terms_list):
        return 0
    else:
        freq = inv_index_frequency[term]
        return freq

In [10]:
def retreive_term_info(inv_index_postings, inv_index_frequency, term):
    terms_list = inv_index_postings.keys()
    if(term not in terms_list):
        return [], 0
    else:
        posting = inv_index_postings[term]
        freq = inv_index_frequency[term]
        return posting, freq

In [11]:
def query_AND(posting1, posting2, verbose=False):


    if(posting1 == [] or posting2 == []):
        return 0, 0, []

    ptr1 = 0
    ptr2 = 0
    answer_docID = []

    num_comparisons = 0

    while(ptr1 < len(posting1) and ptr2 < len(posting2)):
        num_comparisons += 1
        # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")

        if(posting1[ptr1] == posting2[ptr2]):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif(posting1[ptr1] < posting2[ptr2]):
            ptr1 += 1
        else:
            ptr2 += 1

    num_docs_retreived = len(answer_docID)
    doc_names_retreived = getDocsFromID(docID_to_doc_mapping, answer_docID)

    if(verbose==True):
        print(f"AND Query: \nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
    
    # temp_verification = list(set(posting1) & set(posting2))
    # print(f"Verification: {check_equal_arrays(temp_verification, answer_docID)}")

    return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

def query_OR(posting1, posting2, verbose=False):
    

    if((posting1 == []) and (posting2 == [])):
        return 0, 0, []
    elif((posting1 == []) and (posting2 != [])):
        ans_docs = posting2
        return len(ans_docs), 0, ans_docs
    elif((posting1 != []) and (posting2 == [])):
        ans_docs = posting1
        return len(ans_docs), 0, ans_docs
    else:
        ptr1 = 0
        ptr2 = 0
        answer_docID = []

        num_comparisons = 0
        
        while(ptr1 < len(posting1) and ptr2 < len(posting2)):
            num_comparisons += 1
            # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")

            if(posting1[ptr1] == posting2[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
                ptr2 += 1
            elif(posting1[ptr1] < posting2[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
            else:
                answer_docID.append(posting2[ptr2])
                ptr2 += 1
        while(ptr1 < len(posting1)):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
        while(ptr2 < len(posting2)):
            answer_docID.append(posting2[ptr2])
            ptr2 += 1


        num_docs_retreived = len(answer_docID)
        doc_names_retreived = getDocsFromID(docID_to_doc_mapping, answer_docID)

        if(verbose==True):
            print(f"Query OR\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
        
        # temp_verification = list(set(posting1) | set(posting2))
        # print(f"Verification: {check_equal_arrays(temp_verification, answer_docID)}")
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

def perform_NOT(posting):

    all_docIDs = [docID for docID in range(len(file_names))]
    if(posting == []):
        return all_docIDs
    for docID in posting:
        # print(f"Removing docID : {docID}")
        all_docIDs.remove(docID)

    return all_docIDs

def query_AND_NOT(posting1, posting2, verbose=False):

    if((posting1 == [])):
        return 0, 0, []
    
    posting2_NOT = perform_NOT(posting2)
    ptr1 = 0
    ptr2 = 0
    answer_docID = []

    num_comparisons = 0

    while(ptr1 < len(posting1) and ptr2 < len(posting2_NOT)):
        num_comparisons += 1
        # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")
        if(posting1[ptr1] == posting2_NOT[ptr2]):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif(posting1[ptr1] < posting2_NOT[ptr2]):
            ptr1 += 1
        else:
            ptr2 += 1

    num_docs_retreived = len(answer_docID)
    doc_names_retreived = getDocsFromID(docID_to_doc_mapping, answer_docID)

    if(verbose==True):
        print(f"Query AND NOT\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
    
    # temp_verification = list(set(posting1) | set(posting2))
    # print(f"Verification: {check_equal_arrays(temp_verification, answer_docID)}")
    return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

def query_OR_NOT(posting1, posting2, verbose=False):
    
    if((posting1 == []) and (posting2 == [])):
        ans_docs = perform_NOT(posting2)
        return len(ans_docs), 0, ans_docs
    elif((posting1 == []) and (posting2 != [])):
        ans_docs = perform_NOT(posting2)
        return len(ans_docs), 0, ans_docs
    elif((posting1 != []) and (posting2 == [])):
        ans_docs = posting1
        return len(ans_docs), 0, ans_docs
    else:
        
        posting2_NOT = perform_NOT(posting2)
        ptr1 = 0
        ptr2 = 0
        answer_docID = []

        num_comparisons = 0

        while(ptr1 < len(posting1) and ptr2 < len(posting2_NOT)):
            num_comparisons += 1
            # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")
            if(posting1[ptr1] == posting2_NOT[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
                ptr2 += 1
            elif(posting1[ptr1] < posting2_NOT[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
            else:
                answer_docID.append(posting2_NOT[ptr2])
                ptr2 += 1
        while(ptr1 < len(posting1)):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
        while(ptr2 < len(posting2_NOT)):
            answer_docID.append(posting2_NOT[ptr2])
            ptr2 += 1


        num_docs_retreived = len(answer_docID)
        doc_names_retreived = getDocsFromID(docID_to_doc_mapping, answer_docID)

        if(verbose==True):
            print(f"Query OR NOT\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
        
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

In [13]:
def process_ops(ops_string):
    ops = ops_string.split(",")
    ops = [op.strip() for op in ops]
    for op in ops:
        if(not(op == "AND" or op == "OR" or op == "OR NOT" or op == "AND NOT")):
            return []
    return ops

def perform_query(posting1, posting2, op):
    if(op == "AND"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_AND(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    elif(op == "OR"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_OR(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    elif(op == "AND NOT"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_AND_NOT(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    elif(op == "OR NOT"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_OR_NOT(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    else:
        print("INVALID OPERATOR")
        return []

def process_query(inv_index, query_tokens, ops):
    # query_tokens = preprocess_query(query_text)
    print(f"Query tokens : {query_tokens}\n")
    print(f"Operators : ")
    n_qtoks = len(query_tokens)
    n_ops = n_qtoks - 1
    # query_operato = copy.deepcopy(ops)
    if(len(ops) > n_ops):
        print(f"Insufficient tokens in the query\n")
        return -1, -1, []
    elif(len(ops) < n_ops):
        print(f"Insufficient operators")
        return -1, -1, []
    else:
        total_comparisons = 0
        if(n_ops == 1):
            p1 = retreive_postings(inv_index, query_tokens[0])
            p2 = retreive_postings(inv_index,query_tokens[1])
            num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = perform_query(p1, p2, ops[0])
            return num_docs_retreived, num_comparisons, doc_names_retreived
        else:
            p1 = retreive_postings(inv_index, query_tokens[0])
            p2 = retreive_postings(inv_index, query_tokens[1])
            ndocs, total_comparisons, doc_names, ans = perform_query(p1, p2, ops[0])
            qtok_ptr = 2
            for i in range(1, len(ops)):
                posting_tok = retreive_postings(inv_index, query_tokens[qtok_ptr])
                ndocs, cmps, doc_names, ans = perform_query(ans, posting_tok, ops[i])
                total_comparisons += cmps
                qtok_ptr += 1
            num_docs_retreived = len(ans)
            doc_names_retreived = getDocsFromID(docID_to_doc_mapping, ans)
            return num_docs_retreived, total_comparisons, doc_names_retreived

In [14]:
inverted_index_postings, inverted_index_frequency = create_inverted_index(file_toks)

In [17]:
input_query_sequence = input("Enter the space-separated query terms : ")
input_operator_sequence = input("Enter the comma-separated operator sequence enclosed in '[]' : ")
print(f"Input query sequence : {input_query_sequence}\n")
print(f"Input operator sequence : {input_operator_sequence}\n")
input_operators = process_ops(input_operator_sequence[1:len(input_operator_sequence) - 1])
query_tokens = preprocess_query(input_query_sequence)
print(f"Input operator (array) : {input_operators}\n")
print(f"Query tokens : {query_tokens}\n")
query_result_ndocs, query_result_total_cmps, query_result_doc_names = process_query(inverted_index_postings, query_tokens, input_operators)

if(query_result_ndocs != -1):
    query_generated = query_tokens[0] + " " + input_operators[0] + " " + query_tokens[1] + " "
    ptr_qtoks = 2
    for i in range(1, len(input_operators)):
        query_generated += input_operators[i] + " "
        query_generated += query_tokens[ptr_qtoks] + " "
        ptr_qtoks += 1
    print(f"Query : {query_generated}\n\n")
    print(f"\nNo. of documents retreived: {query_result_ndocs}\nNumber of comparisons: {query_result_total_cmps}\nNames of retreived documents: {query_result_doc_names}")



Input query sequence : happy lion a bad boy

Input operator sequence : [OR, OR NOT, AND]

Input operator (array) : ['OR', 'OR NOT', 'AND']

Query tokens : ['happy', 'lion', 'bad', 'boy']

Query tokens : ['happy', 'lion', 'bad', 'boy']

Operators : 
1133
[1, 7, 9, 14, 15, 18, 21, 22, 26, 27, 28, 35, 38, 52, 56, 60, 69, 70, 72, 83, 84, 91, 92, 93, 95, 99, 109, 112, 113, 117, 119, 122, 127, 128, 140, 150, 153, 161, 167, 169, 173, 174, 175, 180, 183, 184, 212, 213, 218, 219, 222, 223, 224, 228, 230, 231, 239, 240, 241, 243, 245, 246, 248, 249, 252, 256, 260, 264, 266, 268, 270, 274, 277, 278, 280, 285, 286, 287, 288, 289, 291, 295, 299, 300, 305, 306, 307, 308, 315, 318, 322, 323, 324, 326, 328, 335, 338, 342, 344, 345, 346, 348, 352, 353, 354, 360, 362, 367, 370, 376, 377, 378, 379, 383, 390, 392, 402, 407, 414, 418, 420, 424, 428, 430, 438, 441, 445, 449, 450, 451, 454, 471, 476, 477, 484, 492, 495, 501, 511, 519, 520, 527, 530, 533, 537, 543, 547, 548, 549, 569, 572, 574, 575, 579, 584,